![](../pictures/PostgresLogo.png)

# Postgres deployment

This time, we will not use the command line but the OpenShift web console to deploy an instance of Postgres. Start by opening the Web interface:

![](../pictures/postgres_1.png)

Now, create a new project:


![](../pictures/postgres_2.png)


For the next step, ensure that you are in the newly created project and add a database deployment template:


![](../pictures/postgres_3.png)


Select the PostgreSQL template:

![](../pictures/postgres_4.png)


![](../pictures/postgres_5.png)


In the next panel, you just need to enter a user and and the password. Leave the rest fields as default:

![](../pictures/postgres_6.png)


After a few minutes, you can check if the deployment succeded by inspectting the pods

In [ ]:
# Replace the command with your own one inside the single quotes and run the cell
# Example OC_LOGIN_COMMAND='oc login --token=sha256~3bR5KXgwiUoaQiph2_kIXCDQnVfm_HQy3YwU2m-UOrs --server=https://c109-e.us-east.containers.cloud.ibm.com:31656'
OC_LOGIN_COMMAND='_replace_this_string_by_pasting_the_clipboard_'
$OC_LOGIN_COMMAND

In [ ]:
# Check if there is one pod running and another one completed
oc project postgres
oc get pods

Check if there is one service defined for postgres

In [ ]:
# Look for a service. The default name will be postgresql and the port 5432
oc get services

In case you want to access the Postgres database with a GUI like [DBeaver](https://dbeaver.io/), [SQuiereL SQL](https://squirrel-sql.sourceforge.io/) or an external python program, it is necessary to externalize the service port:

In [ ]:
servicename=$(oc get services | grep postgresql | awk '{print $1}')
oc expose svc $servicename --type=NodePort --generator="service/v2" --name=pg-nodeport

If everything went well, you will see two services. The port 5432 is the default postgres and it is accessible inside the cluster. The NodePort entry will map 5432 to another one (the number attached right to the colon) and can be used by applications hosted outside OpenShift. 

In [ ]:
# Two services will be displayed:
oc expose svc pg-nodeport
oc get services

And the address of the host can be retrieved by querying the routes:

In [ ]:
# This is the host name of Postgres to access it from outside the cluster
oc get routes

Finally, you can verify if you can connect to the default database and insert data:

In [ ]:
podname=$(oc get pods --field-selector=status.phase=Running -o custom-columns=POD:.metadata.name --no-headers)
oc rsh $podname psql --version
oc rsh $podname psql -c '\l'
oc rsh $podname psql -c 'create table test(c1 integer, c2 char(1));'
oc rsh $podname psql -c "insert into test values(1,'a')"
oc rsh $podname psql -c 'select * from test'


---

Previous Section: [DataStage deployment](./5_datastage_int.ipynb)

[Return to main](../README.md)
